<p align = center>
    <img src="1_Overview.PNG" width=800 height=500 align=center />
<p/>

## Setup

In [5]:
#General
import numpy as np
import scipy
import scipy.linalg
import math

#Molecule Drivers
import pyscf
from pyscf import gto, scf, mcscf

##QISKIT
#Molecule, Fermioninc Operator, Problem Definition
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

#Properties
from qiskit_nature.properties import Property, GroupedProperty
from qiskit_nature.properties.second_quantization.electronic import (
    ElectronicEnergy,
    ElectronicDipoleMoment,
    ParticleNumber,
    AngularMomentum,
    Magnetization,
)
from qiskit_nature.properties.second_quantization.electronic.integrals import (
    ElectronicIntegrals,
    OneBodyElectronicIntegrals,
    TwoBodyElectronicIntegrals,
    IntegralProperty,
)
from qiskit_nature.properties.second_quantization.electronic.bases import ElectronicBasis

#Qubit Operator
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper, BravyiKitaevMapper
from qiskit_nature.converters.second_quantization import QubitConverter

#Circuits, Simulators, Algorithms, Solvers and VQE
from qiskit_nature.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2, TwoLocal, NLocal, PauliTwoDesign
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import VQEUCCFactory, GroundStateEigensolver

##OPENFERMION
from openfermionpyscf import run_pyscf
from openfermion.transforms import binary_code_transform, bravyi_kitaev_code, get_fermion_operator
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.utils import count_qubits
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_ground_state, get_sparse_operator
import numpy
import scipy
import scipy.linalg

##DWAVE
from neal import SimulatedAnnealingSampler
sampler = SimulatedAnnealingSampler()

##UTILITIES
from helper_functions import *
from XBK_method import *
from QCC_method import *


In [93]:
geometry = [['H', [  0.0,    0.0,    0.0]],
            ['H', [  0.0,    0.0,   0.739]]]

# geometry = [['H', [0.0, 0.0, 0.0]], ['H', [1.1, 0.0, 0.0]], ['H', [0.55, 0.9526279441628825, 0]]]

molecule = Molecule(geometry=geometry, charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule=molecule, 
                                           basis='sto6g', 
                                           driver_type=ElectronicStructureDriverType.PYSCF)

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()
fermionic_op = second_q_op[0]
q_fermionin_op_list = fermionic_op.to_list()
print(fermionic_op.to_list()[3])

of_fermionic_op = FermionOperator()
for term in q_fermionin_op_list:
    of_term_coeff = term[1]
    of_term_ops = ""
    for op in range(0,len(term[0]),4):
        q_term_op = term[0][op:op+4]
        if q_term_op[0] == "+":
            of_term_op = q_term_op[2] + "^"
        else:
            of_term_op = q_term_op[2]
        of_term_ops += " " + of_term_op
    of_fermionic_op += FermionOperator(of_term_ops[1:], of_term_coeff)
print(of_fermionic_op)

('-_0 +_1 -_2 +_3', (0.18144182452847724+0j))
(0.18144182452847724+0j) [0 1^ 2 3^] +
(-0.1814418245284773+0j) [0 1^ 2^ 3] +
(-1.258194777661488+0j) [0^ 0] +
(0.4831168317179678+0j) [0^ 0 1^ 1] +
(0.6748905616579757+0j) [0^ 0 2^ 2] +
(0.6645586562464451+0j) [0^ 0 3^ 3] +
(-0.1814418245284773+0j) [0^ 1 2 3^] +
(0.18144182452847726+0j) [0^ 1 2^ 3] +
(-0.47869740107940717+0j) [1^ 1] +
(0.6645586562464451+0j) [1^ 1 2^ 2] +
(0.6994220820957332+0j) [1^ 1 3^ 3] +
(-1.258194777661488+0j) [2^ 2] +
(0.4831168317179678+0j) [2^ 2 3^ 3] +
(-0.47869740107940717+0j) [3^ 3]


In [ ]:
qubit_converter = QubitConverter(JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
# print(qubit_op[4].to_spmatrix())
# qubit_matrix = qubit_op.to_matrix()
# qubit_op_spmatrix = qubit_op.to_spmatrix()

In [22]:
import itertools
import random

def using_tocoo_zip(x):
    cx = x.tocoo()    
    for i,j,v in zip(cx.row, cx.col, cx.data):
        print((i,j),(np.real(v), np.imag(v)))

using_tocoo_zip(qubit_op_spmatrix)

(0, 0) (-8.326672684688674e-17, 0.0)
(1, 1) (-1.2581947776614881, 0.0)
(2, 2) (-0.4786974010794073, 0.0)
(3, 3) (-1.2537753470229274, 0.0)
(4, 4) (-1.258194777661488, 0.0)
(5, 5) (-1.8414989936650004, 0.0)
(5, 10) (0.1814418245284773, 0.0)
(6, 6) (-1.0723335224944501, 0.0)
(6, 9) (0.1814418245284773, 0.0)
(7, 7) (-1.1725209067799944, 0.0)
(8, 8) (-0.4786974010794073, 0.0)
(9, 6) (0.1814418245284773, 0.0)
(9, 9) (-1.0723335224944501, 0.0)
(10, 5) (0.1814418245284773, 0.0)
(10, 10) (-0.25797272006308136, 0.0)
(11, 11) (-0.3684920097601563, 0.0)
(12, 12) (-1.2537753470229271, 0.0)
(13, 13) (-1.1725209067799944, 0.0)
(14, 14) (-0.3684920097601562, 0.0)
(15, 15) (0.19587926220074448, 0.0)


In [ ]:
init_state = HartreeFock(num_SO, particle_number.num_particles, qubit_converter)
# init_state.draw()

# ansatz = UCCSD(qubit_converter, particle_number.num_particles, num_SO,vinitial_state = init_state)
ansatz = TwoLocal(rotation_blocks = ['h', 'rx', 'ry'], entanglement_blocks = 'cz', entanglement='linear', reps=3)
# ansatz.decompose().draw()

numpy_solver = NumPyMinimumEigensolver()
numpy_ground_state_solver = GroundStateEigensolver(qubit_converter, numpy_solver)
numpy_results = numpy_ground_state_solver.solve(es_problem)

exact_energy = numpy_results.computed_energies[0]
# print(f"Exact electronic energy: {exact_energy:.6f} Hartree\n")
# print(numpy_results)

backend = StatevectorSimulator()
optimizer = COBYLA()

error_threshold = 10 # mHartree

np.random.seed(5)  # fix seed for reproducibility
initial_point = np.random.random(ansatz.num_parameters)

# for live plotting
# pp = ProgressPlot(plot_names=['Energy'],
#                   line_names=['Runtime VQE', f'Target + {error_threshold}mH', 'Target']) 
intermediate_info = {
    'nfev': [],
    'parameters': [],
    'energy': [],
    'stddev': []
}

def callback(nfev, parameters, energy, stddev):
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)
    # pp.update([[energy, exact_energy+error_threshold/1000, exact_energy]])

vqe = VQEUCCFactory(backend)
vqe_ground_state_solver = GroundStateEigensolver(qubit_converter, vqe)
vqe_results = vqe_ground_state_solver.solve(es_problem)

# print(vqe_results)

# error = (vqe_results.computed_energies[0] - exact_energy) * 1000 # mHartree
# print(f'Error is: {error:.3f} mHartree')

# TEQUILA!

## Setup

In [66]:
#Execute the following in the terminal to set up conda

# MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
# MINICONDA_PREFIX=/usr/local
# wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
# chmod +x $MINICONDA_INSTALLER_SCRIPT
# ./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [75]:
!pip install tequila-basic

#https://pubs.acs.org/doi/10.1021/acs.jctc.7b00174 <-- whats that?
# !conda install -c conda-forge psi4 -c psi4 --yes

# !python -m pip install --user openfermion
# !python -m pip install --user openfermionpyscf

  Using cached https://files.pythonhosted.org/packages/55/de/bac1892ca411db3934fc62f49d4c2bebe1bf9759bd3584141ed4f6adda98/tequila_basic-1.6.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e7/60/18207ef16aa96ad4491da3e22e93b7eeb64d6a19b22f2d5e04a480aca6e7/cmake-3.21.4-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/72/6c/28e99141edfa6b22126e5d873dd62ce6b9987aebf66d33a98bfe42242f41/openfermion-1.0.0-py3-none-any.whl
  Could not find a version that satisfies the requirement jaxlib (from tequila-basic) (from versions: )
No matching distribution found for jaxlib (from tequila-basic)


## Code

In [77]:
import numpy as np
import tequila as tq
from utility import *

ModuleNotFoundError: No module named 'tequila'

# D-Wave

## Setup

In [78]:
#!curl  -H "X-Auth-Token: YourAPIkey" -X DELETE https://cloud.dwavesys.com/sapi/problems/7211d196-cfc6-4630-84d0-f42ae5d55f0f

#https://www.youtube.com/watch?v=bErs0dxC1aY&list=PLPvKnT7dgEsuhec_yW9oxcZ6gg9SfRFFV
#https://docs.ocean.dwavesys.com/en/latest/overview/install.html#installoceansoftware
#!python -m venv ocean
#!\Scripts\activate

# !pip install dwave-ocean-sdk
# !pip install dwave-qiskit-plugin
# !pip install requests --upgrade --yes
# !dwave setup #API endpoint URL: https://cloud.dwavesys.com/sapi/

#RESTART RUNTIME

## Potentially Useful Junk

In [ ]:
# !pip install pyscf
# !pip install qiskit-nature[pyscf]
# !pip install qiskit

# try:
#     import openfermion
# except ImportError:
#     !pip install -q git+https://github.com/quantumlib/OpenFermion.git@master#egg=openfermion

#with qiskit

#without qiskit (maybe more flexible when working between packages)
# molecule = "H .0 .0 .0; H .0 .0 0.739"
# driver = PySCFDriver(atom=molecule)
# qmolecule = driver.run()
# electronic_energy = qmolecule.get_property(ElectronicEnergy)
# one_body_ints = electronic_energy.get_electronic_integral(ElectronicBasis.MO, 1)
# h1 = qmolecule.one_body_integrals.diagonal()
# h2 = qmolecule.two_body_integrals
# print(h1, h2)

#https://quantumcomputing.stackexchange.com/questions/13803/what-is-the-convention-of-indices-for-the-one-and-two-body-integrals-in-qiskit

# ##some qiskit properties
# properties = driver.run()
# print(properties)
# electronic_energy = properties.get_property(ElectronicEnergy)
# particle_number = properties.get_property(ParticleNumber)
# num_electron = sum(particle_number.num_particles)
# num_MO = num_electron
# num_SO = particle_number.num_spin_orbitals
# num_qubits = particle_number.num_spin_orbitals

# some_properties = {
#     'electronic_energy': electronic_energy.reference_energy,
#     'electrons': num_electron,
#     'MOs': num_MO,
#     'SOs': num_SO,
#     'qubits': num_qubits
# }
# # print(some_properties)